<a href="https://colab.research.google.com/github/ktichola/Image-matching/blob/main/Image_Matching_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'image-matching-challenge-2024:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F71885%2F8143495%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T074754Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4bbfb724ed8889dc846a24ffa97ba5848c163838262af8411e88c7409e5cf323fe6ced8539d00028c0e6c7c5b5bf511230fef7ab1a6e9ac1b688b0d31f2c9b000d04cfaee9b9168ab61c885d8bba5a2d7ba10bc04ee5bc2c70cc1dc6ab1fda75ecef7f0fdbfc29084c4c7f1d738586e1a6891c239c0c0a8fed033e7ee6bf286f7f4dbfbe0f6cab56a307fb0a1faa317afad1c9484413b84e8a23662b34ce8805fde7cb33d6730f5576555490052da484e3edbef1801a904f870d891151f77d964885beb0797411c56bda14146133c1e26eaedb8601baef88fd1da2882204bac858a32f3395309821f022e48258f4562d4c6503579e0a00f653fbf6afbc3ef3b7,imc2024-packages-lightglue-rerun-kornia:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4628051%2F7884485%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T074754Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D749a39809fa7dc0bd4e52f11c199995155eef2c0ba3cae6ce2c0956a1435a0e61c3ff6b996163c0c47108107dad241d7dd17fe951f0915dce63497201877eb05464637b3529907c6f76ddda0f9c00eb646bc30937aadf74a34faede3040e64f68944691468a7fc3dca8ae27b71df25e778fa07b656fb249797a3255d5e68f5123fea4301f163f466783bf80e3e4c994aa013d41decaed2946d1203c6a8ef6177ee2e33dfdc04321d7ff4177f2eeb664480a5e64677d852cadffaf654c8d89f2862071eebbf5c94ce8c85faa1596634a8dbd213a195ffed78cb7cf4053025821781e20d2e18ca188df973f8ca761b1017c97344de5683fd62fbd09d4d7e51f5b0,colmap-db-import:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4628331%2F7884725%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T074754Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0fefc15f9a66f56f557d80242db87585bdbd2aef690342e6e860f54e497d4cb1ce8654583a390d96975cc7388abec80bd92b74448486eeb97b5d830211225f8d11fe1396d073798bc51404f6a148ebbc636b114bff6292e1e527ef0a50aeb02100605d074757c88a13bc6dd99457154901a038ad6322473e7765185638dea388940a3b2ed3a522a36ef51ffcf2055c4b4840be66816dd4d0e8324eb5c061d7692cabbe7c1a07d8714165ac007398e39d4f1968be2aeec97675ec779e2284cd0cba3197aa3a89c1e0f07ea4595384f794cf5cc3612c1772d72e9c7001d0263cf6a8af734f40008e6d315eb84ded535fa0976933efb7c7f359916c4dfd6ba818bf,dinov2/pytorch/base/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F3326%2F4534%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T074754Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8d9204458a8206e812a04ae9f708aaffd0660badc69e60200790ae9fd754001bc7cd745af5a8fd4247dc0ebe24307763f519fb2009e2aea0f7d3c05281f7e5afd70cddda9c8f4f84f6256f18cd4f54ceced0e3e995166bcbca498cd7c3c3cc3841be7c40e2085d4434909ef71b24d13d3f0b454c89d1b6adb917dfa88c9112f0293768004ac2b5111258b596d55e768d2afb57a1454fdd6871fe669256dbd481c08158cd76dbff2872632b61a611a4b7578a1527a74acf2764593fadd54d6862c4e4c6fd843040fa7a30c23e504b86a76044247e1455e30a87446021ce895e44a90c56d7986ec986aa5a4a307e12a97e3453d642dfa928763b5d8e5b341527ba,lightglue/pytorch/aliked/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F14317%2F17191%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T074755Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0de61ce20e0c069ce245417f4e5385f1a6568694a0d37521705ddaccb7be877dbc6bebf01ac6d10e6dd1cf483a3ebca5fe8163f997c010756a6a7495bdc1447a039a95e8a2ab7f9ff3f5ff12276bc7abec46045881079d21fbd53e302ccdef19537326238b333183611490f50a45491390ad6d308581e5bd7d3802c8ef1f81577a6af47a70551b0d1afdd3df305f107d51133340ffd92dbe63f1a6d9ad0b2d25fcd1c6630fa82e6f772b2be02f3506ca433f36b98c217d7e9683d82df6bdb9e7ccf1c0ebfae828f8eac2a896284cfc9454233413b260fdd8a35ff08fed1a2c5ad0e8100a173f9b91a9b429d738186fdfaddbd73b353f21dd5c309e1cbf60415d,aliked/pytorch/aliked-n16/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F14611%2F17555%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T074755Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D65bcfa8d5cd1f91c0025a7870ce2887d85d54b441c898536972f4e46806a37e69d6f4406f0d45cd6611eb863204ce3e5d7d9e4392cbd7523bac2e55315b7152c81ac95682f00e993e26dcee7731bc0ee4cb1e5bdd3bb5a9f26194c3a3f1796c26ffbb1c4e462310e2eece40994dff866c3d35db4445636c74a2a4cc7f2898a8b31cc873c6caa6d7f5332a5e9f6b42af13de3f92bb0822f989e8362c8fc600155c74b69a03acbdd90075c23948560d7e5e618a34df8562ac62eb3fef1b93dd456d5fc7113c6629762259044c975934278125a17cf532b76037fe23de43da1660182fca6be5cfc6ca3b98ba3b2be56898968b2fba42cb87c7386ae92e785f42f9b'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
!pip install --no-index /kaggle/input/imc2024-packages-lightglue-rerun-kornia/* --no-deps
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /kaggle/input/aliked/pytorch/aliked-n16/1/* /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/* /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/aliked_lightglue_v0-1_arxiv-pth

Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia-0.7.2-py2.py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_moons-0.2.9-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_rs-0.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/lightglue-0.0-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/pycolmap-0.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/rerun_sdk-0.15.0a2-cp38-abi3-manylinux_2_31_x86_64.whl
kornia is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
  Attempting uninstall: kornia-rs
    Found existing installation: kornia_rs 0.1.3
    Uninstalling kornia_rs-0.1.3:
      Successfully uninstalled kornia_rs-0.1.3


In [ ]:
import matplotlib.pyplot as plt

import os
from tqdm import tqdm
from pathlib import Path
from time import time, sleep
from fastprogress import progress_bar
import gc
import numpy as np
import h5py
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy
from typing import Any
import itertools
import pandas as pd

# CV/MLe
import cv2
import torch
from torch import Tensor as T
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF
from PIL import Image
from transformers import AutoImageProcessor, AutoModel

import torch
from lightglue import match_pair
from lightglue import LightGlue, ALIKED
from lightglue.utils import load_image, rbd

# 3D reconstruction
import pycolmap

# Data importing into colmap
import sys
sys.path.append("/kaggle/input/colmap-db-import")

# Provided by organizers
from database import *
from h5_to_db import *

def arr_to_str(a):
    """Returns ;-separated string representing the input"""
    return ";".join([str(x) for x in a.reshape(-1)])

def load_torch_image(file_name: Path | str, device=torch.device("cpu")):
    """Loads an image and adds batch dimension"""
    img = K.io.load_image(file_name, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img


def embed_images(
    paths: list[Path],
    model_name: str,
    device: torch.device = torch.device("cpu"),
) :
    """Computes image embeddings.

    Returns a tensor of shape [len(filenames), output_dim]
    """
    processor = AutoImageProcessor.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).eval().to(device)

    embeddings = []

    for i, path in tqdm(enumerate(paths), desc="Global descriptors"):
        image = load_torch_image(path)

        with torch.inference_mode():
            inputs = processor(images=image, return_tensors="pt", do_rescale=False).to(device)
            outputs = model(**inputs) # last_hidden_state and pooled

            # Max pooling over all the hidden states but the first (starting token)
            # To obtain a tensor of shape [1, output_dim]
            # We normalize so that distances are computed in a better fashion later
            embedding = F.normalize(outputs.last_hidden_state[:,1:].max(dim=1)[0], dim=-1, p=2)

        embeddings.append(embedding.detach().cpu())
    return torch.cat(embeddings, dim=0)


def load_torch_image(file_name: Path | str, device=torch.device("cpu")):
    """Loads an image and adds batch dimension"""
    img = K.io.load_image(file_name, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img

def get_image_pairs(paths: list[Path],model_name: str,similarity_threshold: float = 0.6,
                    tolerance: int = 1000,min_matches: int = 20,exhaustive_if_less: int = 20,
                    p: float = 2.0,device: torch.device = torch.device("cpu")):
    # We try to minimize the camera to reconstruct the scene due to saving computing cost and minimizing the error.
    if len(paths) <= exhaustive_if_less:
        return get_pairs_exhaustive(paths)
    matches = []
    embeddings = embed_images(paths, model_name)
    distances = torch.cdist(embeddings, embeddings, p=p)

    # Remove pairs above similarity threshold (if enough)
    mask = distances <= similarity_threshold
    image_indices = np.arange(len(paths))

    for current_image_index in range(len(paths)):
        mask_row = mask[current_image_index]
        indices_to_match = image_indices[mask_row]

        # We don't have enough matches below the threshold, we pick most similar ones
        if len(indices_to_match) < min_matches:
            indices_to_match = np.argsort(distances[current_image_index])[:min_matches]

        for other_image_index in indices_to_match:
            # Skip an image matching itself
            if other_image_index == current_image_index:
                continue

            # We need to check if we are below a certain distance tolerance
            # since for images that don't have enough matches, we picked
            # the most similar ones (which could all still be very different
            # to the image we are analyzing)
            if distances[current_image_index, other_image_index] < tolerance:
                # Add the pair in a sorted manner to avoid redundancy
                matches.append(tuple(sorted((current_image_index, other_image_index.item()))))

    return sorted(list(set(matches)))



2024-05-14 07:42:03.004424: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-14 07:42:03.004545: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-14 07:42:03.171884: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
def detect_keypoints(
    paths: list[Path],
    feature_dir: Path,
    num_features: int = 4096,
    resize_to: int = 1024,
    device: torch.device = torch.device("cpu"),
) -> None:
    """Detects the keypoints in a list of images with ALIKED

    Stores them in feature_dir/keypoints.h5 and feature_dir/descriptors.h5
    to be used later with LightGlue
    """
    dtype = torch.float32 # ALIKED has issues with float16

    extractor = ALIKED(
        max_num_keypoints=num_features,
        detection_threshold=0.01,
        resize=resize_to
    ).eval().to(device, dtype)

    feature_dir.mkdir(parents=True, exist_ok=True)

    with h5py.File(feature_dir / "keypoints.h5", mode="w") as f_keypoints, \
         h5py.File(feature_dir / "descriptors.h5", mode="w") as f_descriptors:

        for path in tqdm(paths, desc="Computing keypoints"):
            key = path.name

            with torch.inference_mode():
                image = load_torch_image(path, device=device).to(dtype)
                features = extractor.extract(image)

                f_keypoints[key] = features["keypoints"].squeeze().detach().cpu().numpy()
                f_descriptors[key] = features["descriptors"].squeeze().detach().cpu().numpy()

In [ ]:
def keypoint_distances(
    paths: list[Path],
    index_pairs: list[tuple[int, int]],
    feature_dir: Path,
    min_matches: int = 15,
    verbose: bool = True,
    device: torch.device = torch.device("cpu"),
) -> None:
    """Computes distances between keypoints of images.

    Stores output at feature_dir/matches.h5
    """

    matcher_params = {
        "width_confidence": -1,
        "depth_confidence": -1,
        "mp": True if 'cuda' in str(device) else False,
    }
    matcher = KF.LightGlueMatcher("aliked", matcher_params).eval().to(device)

    with h5py.File(feature_dir / "keypoints.h5", mode="r") as f_keypoints, \
         h5py.File(feature_dir / "descriptors.h5", mode="r") as f_descriptors, \
         h5py.File(feature_dir / "matches.h5", mode="w") as f_matches:

            for idx1, idx2 in tqdm(index_pairs, desc="Computing keypoing distances"):
                key1, key2 = paths[idx1].name, paths[idx2].name

                keypoints1 = torch.from_numpy(f_keypoints[key1][...]).to(device)
                keypoints2 = torch.from_numpy(f_keypoints[key2][...]).to(device)
                descriptors1 = torch.from_numpy(f_descriptors[key1][...]).to(device)
                descriptors2 = torch.from_numpy(f_descriptors[key2][...]).to(device)

                with torch.inference_mode():
                    distances, indices = matcher(
                        descriptors1,
                        descriptors2,
                        KF.laf_from_center_scale_ori(keypoints1[None]),
                        KF.laf_from_center_scale_ori(keypoints2[None]),
                    )

                # We have matches to consider
                n_matches = len(indices)
                if n_matches:
                    if verbose:
                        print(f"{key1}-{key2}: {n_matches} matches")
                # Store the matches in the group of one image
                group  = f_matches.require_group(key1)
                if n_matches >= min_matches:
                     group.create_dataset(key2, data=indices.detach().cpu().numpy().reshape(-1, 2))

In [ ]:
class Config:
    base_path: Path = Path("/kaggle/input/image-matching-challenge-2024")
    feature_dir: Path = Path.cwd() / ".feature_outputs"
    device: torch.device= K.utils.get_cuda_device_if_available(0)
    pair_matching_args = {
        "model_name": "/kaggle/input/dinov2/pytorch/base/1",
        "similarity_threshold": 0.3,
        "tolerance": 1000,
        "min_matches": 20,
        "exhaustive_if_less": 20,
        "p": 2.0
    }
    keypoint_detection_args = {
        "num_features": 4096,
        "resize_to": 1024,
    }

    keypoint_distances_args = {
        "min_matches": 15,
        "verbose": False,
    }
    colmap_mapper_options = {
        "min_model_size": 3, # By default colmap does not generate a reconstruction if less than 10 images are registered. Lower it to 3.
        "max_num_models": 2,
    }

In [ ]:
def parse_sample_submission(
    base_path: Path,
) -> dict[dict[str, list[Path]]]:
    """Construct a dict describing the test data as

    {"dataset": {"scene": [<image paths>]}}
    """
    data_dict = {}
    with open(base_path / "sample_submission.csv", "r") as f:
        for i, l in enumerate(f):
            # Skip header
            if i == 0:
                print("header:", l)

            if l and i > 0:
                image_path, dataset, scene, _, _ = l.strip().split(',')
                if dataset not in data_dict:
                    data_dict[dataset] = {}
                if scene not in data_dict[dataset]:
                    data_dict[dataset][scene] = []
                data_dict[dataset][scene].append(Path(base_path / image_path))

    for dataset in data_dict:
        for scene in data_dict[dataset]:
            print(f"{dataset} / {scene} -> {len(data_dict[dataset][scene])} images")

    return data_dict

In [ ]:
def create_submission(
    results: dict,
    data_dict: dict[dict[str, list[Path]]],
    base_path: Path,
) -> None:
    """Prepares a submission file."""

    with open("submission.csv", "w") as f:
        f.write("image_path,dataset,scene,rotation_matrix,translation_vector\n")

        for dataset in data_dict:
            # Only write results for datasets with images that have results
            if dataset in results:
                res = results[dataset]
            else:
                res = {}

            # Same for scenes
            for scene in data_dict[dataset]:
                if scene in res:
                    scene_res = res[scene]
                else:
                    scene_res = {"R":{}, "t":{}}

                # Write the row with rotation and translation matrices
                for image in data_dict[dataset][scene]:
                    if image in scene_res:
                        print(image)
                        R = scene_res[image]["R"].reshape(-1)
                        T = scene_res[image]["t"].reshape(-1)
                    else:
                        R = np.eye(3).reshape(-1)
                        T = np.zeros((3))
                    image_path = str(image.relative_to(base_path))
                    f.write(f"{image_path},{dataset},{scene},{arr_to_str(R)},{arr_to_str(T)}\n")

In [ ]:
results = {}
config = Config()
data_dict = parse_sample_submission(config.base_path)
datasets = list(data_dict.keys())


for dataset in datasets:
    if dataset not in results:
        results[dataset] = {}

    for scene in data_dict[dataset]:
        images_dir = data_dict[dataset][scene][0].parent
        results[dataset][scene] = {}
        image_paths = data_dict[dataset][scene]
        print (f"Got {len(image_paths)} images")

        try:
            feature_dir = config.feature_dir / f"{dataset}_{scene}"
            feature_dir.mkdir(parents=True, exist_ok=True)
            database_path = feature_dir / "colmap.db"
            if database_path.exists():
                database_path.unlink()

            # 1. Get the pairs of images that are somewhat similar
            index_pairs = get_image_pairs(
                image_paths,
                **config.pair_matching_args,
                device=config.device,
            )
            gc.collect()

            # 2. Detect keypoints of all images
            detect_keypoints(
                image_paths,
                feature_dir,
                **config.keypoint_detection_args,
                device=config.device,
            )
            gc.collect()

            # 3. Match  keypoints of pairs of similar images
            keypoint_distances(
                image_paths,
                index_pairs,
                feature_dir,
                **config.keypoint_distances_args,
                device=config.device,
            )
            gc.collect()

            sleep(1)

            # 4.1. Import keypoint distances of matches into colmap for RANSAC
            import_into_colmap(
                images_dir,
                feature_dir,
                database_path,
            )

            output_path = feature_dir / "colmap_rec_aliked"
            output_path.mkdir(parents=True, exist_ok=True)

            # 4.2. Compute RANSAC (detect match outliers)
            # By doing it exhaustively we guarantee we will find the best possible configuration
            pycolmap.match_exhaustive(database_path)

            mapper_options = pycolmap.IncrementalPipelineOptions(**config.colmap_mapper_options)

            # 5.1 Incrementally start reconstructing the scene (sparse reconstruction)
            # The process starts from a random pair of images and is incrementally extended by
            # registering new images and triangulating new points.
            maps = pycolmap.incremental_mapping(
                database_path=database_path,
                image_path=images_dir,
                output_path=output_path,
                options=mapper_options,
            )

            print(maps)
            clear_output(wait=False)

            # 5.2. Look for the best reconstruction: The incremental mapping offered by
            # pycolmap attempts to reconstruct multiple models, we must pick the best one
            images_registered  = 0
            best_idx = None

            print ("Looking for the best reconstruction")

            if isinstance(maps, dict):
                for idx1, rec in maps.items():
                    print(idx1, rec.summary())
                    try:
                        if len(rec.images) > images_registered:
                            images_registered = len(rec.images)
                            best_idx = idx1
                    except Exception:
                        continue

            # Parse the reconstruction object to get the rotation matrix and translation vector
            # obtained for each image in the reconstruction
            if best_idx is not None:
                for k, im in maps[best_idx].images.items():
                    key = config.base_path / "test" / scene / "images" / im.name
                    results[dataset][scene][key] = {}
                    results[dataset][scene][key]["R"] = deepcopy(im.cam_from_world.rotation.matrix())
                    results[dataset][scene][key]["t"] = deepcopy(np.array(im.cam_from_world.translation))

            print(f"Registered: {dataset} / {scene} -> {len(results[dataset][scene])} images")
            print(f"Total: {dataset} / {scene} -> {len(data_dict[dataset][scene])} images")
            create_submission(results, data_dict, config.base_path)
            gc.collect()

        except Exception as e:
            print(e)

Looking for the best reconstruction
0 Reconstruction:
	num_reg_images = 38
	num_cameras = 38
	num_points3D = 12711
	num_observations = 65700
	mean_track_length = 5.16875
	mean_observations_per_image = 1728.95
	mean_reprojection_error = 0.816243
Registered: church / church -> 38 images
Total: church / church -> 41 images
/kaggle/input/image-matching-challenge-2024/test/church/images/00046.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00090.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00092.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00087.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00050.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00068.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00083.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00096.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00069.png
/kaggle/input/image-m